In [2]:
#  TDHPCWL40AZFBJ82
def get_financial(symbol):
    import requests
    import pandas as pd
    import time
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url_earning = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+symbol+'&apikey=TDHPCWL40AZFBJ82'
    url_balance_sheet = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+symbol+'&apikey=TDHPCWL40AZFBJ82'
    url_income_statement = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+symbol+'&apikey=TDHPCWL40AZFBJ82'

    r_earning = requests.get(url_earning)
    r_balance_sheet = requests.get(url_balance_sheet)
    r_income_statement = requests.get(url_income_statement)
    

    data_earning = r_earning.json()
    data_balance_sheet = r_balance_sheet.json()
    data_income_statement = r_income_statement.json()

    df_earning = pd.DataFrame(data_earning["quarterlyEarnings"])
    df_earning = df_earning[['fiscalDateEnding','reportedEPS']].drop(index=0).reset_index(drop=True).head(20)

    df_balance = pd.DataFrame(data_balance_sheet["quarterlyReports"])
    df_balance = df_balance[['fiscalDateEnding','totalAssets','totalLiabilities']]

    df_income_statement = pd.DataFrame(data_income_statement["quarterlyReports"])
    df_income_statement = df_income_statement[['fiscalDateEnding','grossProfit','totalRevenue','netIncome']]

    df = df_earning.join(df_balance.set_index('fiscalDateEnding'), on='fiscalDateEnding').join(df_income_statement.set_index('fiscalDateEnding'), on='fiscalDateEnding')
    df.dropna(inplace=True)
    print(df)
    row_lists = df.apply(lambda row: row.tolist(), axis=1).tolist()
    return row_lists


In [4]:
import sqlite3
import time
def insert_financial(finance,stock_id):
    from sqlite3 import connect
    conn = connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute(f"INSERT INTO nasdaq_financial_statement VALUES (null,{stock_id},'{finance[0]}',{finance[1]},{finance[2]},{finance[3]},{finance[4]},{finance[5]},{finance[6]})")
    conn.commit()#commit change to db
    conn.close()#disconnect
def get_stock_id(symbol):
    """return id of stock"""       
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT stock_id FROM stock_nasdaq WHERE symbol = '{symbol}'")
    data = cursor.fetchall()
    conn.close()
    return data[0][0]

def get_all_stock():
    import sqlite3
    """return list of symbol of stock"""
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT symbol FROM stock_nasdaq")
    data = cursor.fetchall()
    conn.close()
    return [i[0] for i in data]

from data.function import Stock,Categories
stock = ['AAPL','ABBV','AMZN','ASML','AVGO','AZN','BABA','BAC','COST','CSCO','CVX','GOOG','GOOGL','HD','JNJ','JPM','KO','LLY','MA','META','MRK','MSFT','NVDA','NVO','ORCL','PCAR','PEP','PFE','PG','SHEL','TMO','TSLA','TSM','UNH','V','WMT','XOM']
cont = False
for i in stock:
    if cont == True:
        print(i)
        id = get_stock_id(i)
        financial = get_financial(i)
        for j in financial:
            insert_financial(j,id)
        time.sleep(61)
    if i == 'BRK':
        cont = True

COST
Empty DataFrame
Columns: [fiscalDateEnding, reportedEPS, totalAssets, totalLiabilities, grossProfit, totalRevenue, netIncome]
Index: []
CSCO
  fiscalDateEnding reportedEPS  totalAssets totalLiabilities grossProfit  \
2       2022-04-30        0.87  92797000000      52397000000  8121000000   
5       2021-07-31        0.84  97497000000      56222000000  8344000000   

  totalRevenue   netIncome  
2  12762000000  3044000000  
5  13091000000  3009000000  
CVX
   fiscalDateEnding reportedEPS   totalAssets totalLiabilities  grossProfit  \
0        2022-09-30        5.56  259735000000     100108000000  27318000000   
1        2022-06-30        5.82  257936000000     103374000000  27909000000   
2        2022-03-31        3.36  249048000000     101948000000  20693000000   
3        2021-12-31        2.56  239535000000      99595000000  17785000000   
4        2021-09-30        2.96  239948000000     103226000000  20119000000   
5        2021-06-30        1.71  242806000000     1088950000